In [1]:
import sys 
sys.path.append('..')

#Dependencies

import os
from tqdm import tqdm
import pandas as pd
import numpy as np

import json 
import pickle

from src.environment.ml_env import SimulatedEnv, SimulatedFairEnv
from src.model.recommender import DRRAgent, FairRecAgent

In [2]:
dataset_path = "../data/movie_lens_100k_output_path.json"
with open(dataset_path) as json_file:
    _dataset_path = json.load(json_file)


dataset = {}
with open(os.path.join("..", _dataset_path["eval_users_dict"]), "rb") as pkl_file:
    dataset["eval_users_dict"] = pickle.load(pkl_file)

with open(os.path.join("..", _dataset_path["eval_users_dict_positive_items"]), "rb") as pkl_file:
    dataset["eval_users_dict_positive_items"] = pickle.load(pkl_file)

with open(os.path.join("..", _dataset_path["eval_users_history_lens"]), "rb") as pkl_file:
    dataset["eval_users_history_lens"] = pickle.load(pkl_file)

with open(os.path.join("..", _dataset_path["users_history_lens"]), "rb") as pkl_file:
    dataset["users_history_lens"] = pickle.load(pkl_file)

with open(os.path.join("..", _dataset_path["movies_groups"]), "rb") as pkl_file:
    dataset["movies_groups"] = pickle.load(pkl_file)

In [3]:
ENV = dict(drr=SimulatedEnv, fairrec=SimulatedFairEnv)
AGENT = dict(drr=DRRAgent, fairrec=FairRecAgent)

In [4]:
train_ids = [
    "movie_lens_100k_2021-10-24_01-42-57", # long training
    "movie_lens_100k_fair_2021-10-24_01-41-02", # long training
]

In [5]:
algorithm = "drr"
train_version = "movie_lens_100k"
train_id = train_ids[0]
output_path = "../model/{}/{}".format(train_version, train_id)

users_num = 943
items_num = 1682

state_size = 5
srm_size = 3

embedding_dim = 50
actor_hidden_dim = 512
actor_learning_rate = 0.0001
critic_hidden_dim = 512
critic_learning_rate = 0.001
discount_factor = 0.9
tau = 0.01
learning_starts = 100
replay_memory_size = 1000000
batch_size = 1
emb_model = "user_movie"
embedding_network_weights = "../model/pmf/emb_50_ratio_0.800000_bs_1000_e_258_wd_0.100000_lr_0.000100_trained_pmf.pt"
n_groups = 10
fairness_constraints = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


top_k = None
done_count = 10

In [6]:
actor_checkpoint = sorted(
    [
        int((f.split("_")[1]).split(".")[0])
        for f in os.listdir(output_path)
        if f.startswith("actor_")
    ]
)[-1]
critic_checkpoint = sorted(
    [
        int((f.split("_")[1]).split(".")[0])
        for f in os.listdir(output_path)
        if f.startswith("critic_")
    ]
)[-1]

print(actor_checkpoint, critic_checkpoint)

30000 30000


In [7]:
# # with open(os.path.join("../model/movie_lens_100k/movie_lens_100k_2021-10-26_11-45-48/buffer.pkl"), "rb") as pkl_file:
# #     buffer_load = pickle.load(pkl_file)

# with open(os.path.join("../model/movie_lens_100k_fair/movie_lens_100k_fair_2021-10-26_11-45-51/buffer.pkl"), "rb") as pkl_file:
#     buffer_load = pickle.load(pkl_file)

In [8]:
_precision = []
_propfair = []
_reward = []
_ufg = []
for i in range(10):
    sum_precision = 0
    sum_propfair = 0
    sum_reward = 0

    env = ENV[algorithm](
        users_dict=dataset["eval_users_dict"],
        users_history_lens=dataset["eval_users_history_lens"],
        n_groups=n_groups,
        movies_groups=dataset["movies_groups"],
        state_size=state_size,
        done_count=done_count,
        fairness_constraints=fairness_constraints,
    )
    available_users = env.available_users

    recommender = AGENT[algorithm](
        env=env,
        users_num=users_num,
        items_num=items_num,
        genres_num=0,
        movies_genres_id={}, 
        srm_size=srm_size,
        state_size=state_size,
        train_version=train_version,
        is_test=True,
        embedding_dim=embedding_dim,
        actor_hidden_dim=actor_hidden_dim,
        actor_learning_rate=actor_learning_rate,
        critic_hidden_dim=critic_hidden_dim,
        critic_learning_rate=critic_learning_rate,
        discount_factor=discount_factor,
        tau=tau,
        learning_starts=learning_starts,
        replay_memory_size=replay_memory_size,
        batch_size=batch_size,
        model_path=output_path,
        emb_model=emb_model,
        embedding_network_weights_path=embedding_network_weights,
        n_groups=n_groups,
        fairness_constraints=fairness_constraints,
    )

    for user_id in tqdm(available_users):
        recommender.load_model(
            os.path.join(output_path, "actor_{}.h5".format(actor_checkpoint)),
            os.path.join(
                output_path, "critic_{}.h5".format(critic_checkpoint)
            ),
        )
        # recommender.buffer = buffer_load

        eval_env = ENV[algorithm](
            users_dict=dataset["eval_users_dict"],
            users_history_lens=dataset["eval_users_history_lens"],
            n_groups=n_groups,
            movies_groups=dataset["movies_groups"],
            state_size=state_size,
            done_count=done_count,
            fairness_constraints=fairness_constraints,
            fix_user_id=user_id,
            reward_model=recommender.reward_model,
            device=recommender.device,
        )

        recommender.env = eval_env

        precision, ndcg, propfair, reward = recommender.train(
            max_episode_num=1, top_k=top_k
        )

        sum_precision += precision
        sum_propfair += propfair
        sum_reward += reward

        del eval_env


    _precision.append(sum_precision / len(dataset["eval_users_dict"]))
    _propfair.append(sum_propfair / len(dataset["eval_users_dict"]))
    _ufg.append((sum_propfair / len(dataset["eval_users_dict"]))
        / (1 - (sum_precision / len(dataset["eval_users_dict"]))))
    _reward.append(sum_reward)

    print("---------- Evaluation")
    print("- sum reward: ", round(sum_reward, 2))
    print("- precision@: ", round(sum_precision / len(dataset["eval_users_dict"]), 4))
    print("- propfair: ", round(sum_propfair / len(dataset["eval_users_dict"]), 4)), 
    print(
        "- ufg: ",
        round((sum_propfair / len(dataset["eval_users_dict"]))
        / (1 - (sum_precision / len(dataset["eval_users_dict"]))), 4)
    )
    print()


print(round(np.mean(_propfair), 4))
print(round(np.mean(_precision), 4))
print(round(np.mean(_reward), 4))
print(round(np.mean(_ufg), 4))

100%|██████████| 185/185 [00:19<00:00,  9.45it/s]


---------- Evaluation
- sum reward:  480.1
- precision@:  0.8161
- propfair:  0.8535
- ufg:  4.6397



100%|██████████| 185/185 [00:19<00:00,  9.45it/s]


---------- Evaluation
- sum reward:  492.33
- precision@:  0.8124
- propfair:  0.8598
- ufg:  4.5836



100%|██████████| 185/185 [00:18<00:00,  9.92it/s]


...

---------- Evaluation
- sum reward:  505.26
- precision@:  0.8198
- propfair:  0.8604
- ufg:  4.7754



100%|██████████| 185/185 [00:18<00:00,  9.86it/s]


---------- Evaluation
- sum reward:  468.2
- precision@:  0.8013
- propfair:  0.8493
- ufg:  4.2745



100%|██████████| 185/185 [00:18<00:00,  9.86it/s]

---------- Evaluation
- sum reward:  479.03
- precision@:  0.8103
- propfair:  0.8553
- ufg:  4.5076

0.855
0.8116
481.109
4.5454


In [ ]:
print(round(np.mean(_propfair), 4))
print(round(np.mean(_precision), 4))
print(round(np.mean(_reward), 4))
print(round(np.mean(_ufg), 4))

In [9]:
_propfair

[0.8534648878974421,
 0.8597976954985628,
 0.8550059617766729,
 0.8496948042935616,
 0.8573013600447137,
 0.8551669763910876,
 0.8550675940525352,
 0.8603760957875722,
 0.8493064771195712,
 0.8553115818598706]

In [ ]:
_precision

In [ ]:
_reward

In [ ]:
_ufg

In [12]:
print(round(_propfair[3], 4))
print(round(_precision[3], 4))
print(round(_reward[3], 4))
print(round(_ufg[3], 4))

0.8497
0.8142
470.7087
4.5724
